In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
rds_connection_string = "postgres:postgres@localhost:5432/ETLProject"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
engine.table_names()

['cities', 'lines', 'stations', 'stationlines', 'systems', 'tracks']

In [6]:
csv_file = "./DataSources/stations.csv"
stations_data_df = pd.read_csv(csv_file)
stations_data_df.head(100)

,id,name,geometry,buildstart,opening,closure,city_id
0,7694,Keisei Tsudanuma,POINT(140.024812197129 35.6837744784723),1921.0,1921.0,999999.0,114
1,6003,Kossuth Lajos tér,POINT(19.0462376564033 47.5054880717671),0.0,0.0,999999.0,29
2,7732,Saint-Charles,POINT(5.3801556 43.3024646),1973.0,1977.0,999999.0,74
3,7695,Keisei Makuhari-Hongo,POINT(140.042146725175 35.6726021159981),1991.0,1991.0,999999.0,114
4,7726,Chartreux,POINT(5.4014815 43.309129),1973.0,1977.0,999999.0,74
5,7696,Keisei Makuhari,POINT(140.056077093286 35.6605591225961),1921.0,1921.0,999999.0,114
6,7719,Malpassé,POINT(5.4165033 43.3209501),1973.0,1977.0,999999.0,74
7,7697,Kemigawa,POINT(140.066304589107 35.6526280375642),1921.0,1921.0,999999.0,114
8,1,Caseros,POINT(-58.3989075634122 -34.6358418393779),2001.0,2007.0,NaN,1
9,7698,Keisei Inage,POINT(140.085607402414 35.6378035803563),1921.0,1921.0,999999.0,114
